# GET DATA FROM ALPHA VANTAGE

In [ ]:
from datetime import datetime, timedelta
import os

import requests
import pandas as pd

In [21]:
KEY2 = os.getenv("HSTH_AV_KEY")


SOURCE="alphavantage"
API_KEY = KEY2
YYYY="2000"
MM="01"
INTERVAL="5min"
TICKER="SPY"

In [ ]:
def get_historicals_from_alphavantage():
    print("getting data")

    # url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=SPY&interval=5min&apikey={API_KEY}"
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={TICKER}&interval={INTERVAL}&month={YYYY}-{MM}&outputsize=full&apikey={API_KEY}"
    response = requests.get(url)
    data = response.json()
    return data

In [ ]:
data = get_historicals_from_alphavantage()

looking to get data


In [19]:
df = pd.DataFrame.from_dict(data["Time Series (5min)"], orient="index")
print(df.head())


                     1. open  2. high   3. low 4. close 5. volume
2000-01-31 16:30:00  88.8758  88.8758  88.8758  88.8758       100
2000-01-31 16:20:00  88.9156  88.9156  88.9156  88.9156      2000
2000-01-31 16:15:00  88.8758  88.8758  88.8758  88.8758     20500
2000-01-31 16:10:00  88.8957  88.9355  88.7763  88.8758     77600
2000-01-31 16:05:00  88.7763  88.8559  88.7564  88.8360     99300


In [26]:
csv_name = f"{YYYY}-{MM}-{TICKER}_{INTERVAL}_{SOURCE}.csv"

In [27]:

df.to_csv(f"../data/{csv_name}")

# UPLOAD DATA TO GOOGLE CLOUD BUCKET

In [28]:
from google.cloud import storage


In [ ]:


def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # The ID of your GCS bucket:        bucket_name = "your-bucket-name"
    # The path to your file to upload:  source_file_name = "local/path/to/file"
    # The ID of your GCS object:        destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    # Optional: set a generation-match precondition to avoid potential race conditions
    # and data corruptions. The request to upload is aborted if the object's
    # generation number does not match your precondition. For a destination
    # object that does not yet exist, set the if_generation_match precondition to 0.
    # If the destination object already exists in your bucket, set instead a
    # generation-match precondition using its generation number.
    generation_match_precondition = 0

    blob.upload_from_filename(source_file_name, if_generation_match=generation_match_precondition)

    print(
        f"File {source_file_name} uploaded to {destination_blob_name}."
    )

In [37]:
bucket_name = "cgpdata"
source_file_name = f"../data/{csv_name}"
destination_blob_name = f"raw/{csv_name}"

In [38]:
upload_blob(bucket_name, source_file_name, destination_blob_name)

File ../data/2000-01-SPY_5min_alphavantage.csv uploaded to raw/2000-01-SPY_5min_alphavantage.csv.
